In [1]:
use Dan :ALL;

In [2]:
# Read .csv file (Text::CSV)

use Text::CSV;
my @lines  = csv( in => 'DemographicData.csv' );

my \stats = DataFrame.new( columns => @lines[0], data => @lines[1..*-1] );
~stats[0..5]^;      # head

    Country Name          Country Code  Birth rate  Internet users  Income Group        
 0  Aruba                 ABW           10.244      78.9            High income         
 1  Afghanistan           AFG           35.253      5.9             Low income          
 2  Angola                AGO           45.985      19.1            Upper middle income 
 3  Albania               ALB           12.877      57.2            Upper middle income 
 4  United Arab Emirates  ARE           11.044      88              High income         
 5  Argentina             ARG           17.716      59.9            High income         

These two read csv options illustrate how the close relationship between raku code, modules and Dan objects makes it easy for programmers to unpack what is going on and to step in. 

What if you encounter non-standard character "noise" in input files, or non-standard quoting constructs? ... just write your own csv Regex - or even a whole Grammar! Also note the use of the << $_ >> which turns a Str into an [Allomorph](https://docs.raku.org/syntax/%3C%3C%20%3E%3E) such as IntStr, RatStr, NumStr...

What if you want RFC4180 compliance or unicode separators or unusual eol markers? ... then the comprehensive raku [Text::CSV](https://github.com/Tux/CSV) module is for you.

In [ ]:
# Read .csv file (Regex)

#| comb match anything except " and , -or- take a quoted string
my Regex \csv = rx/ <-[",]>+ | <["]> ~ <["]>  <-["]>+ /;

my @lines   = 'DemographicData.csv'.IO.lines;

my @columns = @lines.shift.comb( csv );
my @data    = @lines.map(*.comb( csv ).map({<< $_ >>}).Array);

my \stats2   = DataFrame.new( :@columns, :@data );
~stats2[0..5]^;      # head

In [ ]:
# '~' makes a Str, '^' makes a DataFrame from rows

~stats[*-5..*-1]^;    # tail

In [ ]:
stats.shape;

In [ ]:
stats.elems;

In [ ]:
stats.columns.elems;  

In [ ]:
#Clean up #1 - tidy up col labels

stats.columns = <CountryName CountryCode BirthRate InternetUsers IncomeGroup> Z=> 0..∞;
say stats.columns;     #Hash (unordered)
say stats.cx;          #Seq    (ordered)

In [ ]:
#Clean up #2 - coerce cols to numbers      [see DD-raku-Dan-typed.ipynb for why & when]

stats.data[*;2,3].map({$_.=Rat});
stats.dtypes;

In [ ]:
~stats.describe;

In [ ]:
~stats[0..5]<CountryName BirthRate>;

In [ ]:
~stats[185..*-1]^;       # Range operator (start .. end)

In [ ]:
#Reverse order
~stats[10,*-1...5]^;     # Sequence operator (start,operation ... end)

In [ ]:
#Every 20th item
~stats[0,20...stats.elems]^;

In [ ]:
#Calculate and Append a new column

my $calcs = Series.new( stats[*]<BirthRate> >>*<< stats[*]<InternetUsers> );   #multiply every row

stats.splice( :ax, *, *, [MyCalc => $calcs] );
~stats[0..5]^;

In [ ]:
#Drop column

stats.splice( :ax, *-1 );
~stats[0..5]^;

In [ ]:
# Grep (check whether column 3 value < 2)

~stats.grep( { .[3] < 2 } );

In [ ]:
# Grep (simple col label check)

~stats.grep( { stats[$++]<InternetUsers> < 2 } );

In [ ]:
# Grep (combine two checks with col labels)

~stats.grep( { my $i=stats[$++]; $i.<InternetUsers> < 2 && $i.<BirthRate> > 40 } );

In [ ]:
# Grep (define filters and apply later)

sub filter1 { $^i.<InternetUsers> < 2  };
sub filter2 { $^i.<BirthRate>     > 40 };

~stats.grep( { my $i=stats[$++]; $i.&filter1 && $i.&filter2 } );

In [ ]:
# Grep with regex

stats.grep( { stats[$++]<IncomeGroup> ~~ /High/ } ).elems;

In [ ]:
# Unique

stats[*]<IncomeGroup>.unique.join(', ');

In [ ]:
# Accessors

stats[4][3];

In [ ]:
stats[4]<InternetUsers>;

In [ ]:
stats<4>[3];   #The default index is letter 4 => 4 (Int) so you may use either [] or <> accessor